# Test RLS on account table

In [ ]:
import os

os.environ["LAMIN_ENV"]

In [ ]:
import pytest
from laminhub_rest.connector import connect_hub, connect_hub_with_auth
from laminhub_rest.utils._test import create_test_auth, create_test_account
from laminhub_rest.core.account._crud import (
    sb_insert_account,
    sb_update_account,
    sb_select_account_by_handle,
    sb_delete_account,
)
import string, secrets
from laminhub_rest.core.account import delete_account
from laminhub_rest._clean_ci import clean_ci

In [ ]:
def base62(n_char: int) -> str:
    """Like nanoid without hyphen and underscore."""
    alphabet = string.digits + string.ascii_letters.swapcase()
    id = "".join(secrets.choice(alphabet) for i in range(n_char))
    return id

## Parametrize 

In [ ]:
hub = connect_hub()

# Create account 1
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_id_1 = auth_1["id"]
account_handle_1 = auth_1["handle"]
account_hub_1 = connect_hub_with_auth(access_token=access_token_1)

# Create account 2
auth_2 = create_test_auth()
access_token_2 = auth_2["access_token"]
account_2 = create_test_account(handle=auth_2["handle"], access_token=access_token_2)
account_id_2 = account_2["id"]
account_handle_2 = account_2["handle"]
account_hub_2 = connect_hub_with_auth(access_token=access_token_2)

## Enable everyone to see an account

In [ ]:
account = sb_select_account_by_handle(account_handle_2, hub)

assert account is not None

## Enable accounts to create a corresponding entry in account table

With an anonymous Supabase client

In [ ]:
with pytest.raises(Exception) as error:
    account = sb_insert_account(
        account_fields={
            "id": account_id_1,
            "user_id": account_id_1,
            "lnid": base62(8),
            "handle": account_handle_1,
        },
        supabase_client=hub,
    )


assert "new row violates row-level security policy" in error.value.message

Using corresponding account authentication

In [ ]:
account = sb_insert_account(
    account_fields={
        "id": account_id_1,
        "user_id": account_id_1,
        "lnid": base62(8),
        "handle": account_handle_1,
    },
    supabase_client=account_hub_1,
)

assert account is not None

## Enable accounts to update their data

Using corresponding account authentication

In [ ]:
account = sb_update_account(
    account_id=account_id_1,
    account_fields={"name": "Local test user 2 new name"},
    supabase_client=account_hub_1,
)

assert (
    sb_select_account_by_handle(handle=account_handle_1, supabase_client=account_hub_1)[
        "name"
    ]
    == "Local test user 2 new name"
)
assert account is not None

Using another account authentication

In [ ]:
account = sb_update_account(
    account_id=account_id_1,
    account_fields={"name": "Local test user 2 another new name"},
    supabase_client=account_hub_2,
)

assert (
    sb_select_account_by_handle(handle=account_handle_1, supabase_client=account_hub_1)[
        "name"
    ]
    == "Local test user 2 new name"
)
assert account is None

With an anonymous Supabase client

In [ ]:
account = sb_update_account(
    account_id=account_id_1,
    account_fields={"name": "Local test user 2 another new name"},
    supabase_client=hub,
)

assert (
    sb_select_account_by_handle(handle=account_handle_1, supabase_client=hub)["name"]
    == "Local test user 2 new name"
)
assert account is None

## Enable accounts to delete their entry in account table

Using corresponding account authentication

In [ ]:
account = sb_delete_account(handle=account_handle_1, supabase_client=account_hub_1)

assert (
    sb_select_account_by_handle(handle=account_handle_1, supabase_client=account_hub_1)
    is None
)
assert account is not None

Using corresponding account authentication

In [ ]:
account = sb_delete_account(handle=account_handle_2, supabase_client=account_hub_1)

assert (
    sb_select_account_by_handle(handle=account_handle_2, supabase_client=account_hub_2)
    is not None
)
assert account is None

Using corresponding account authentication

In [ ]:
account = sb_delete_account(handle=account_handle_2, supabase_client=hub)

assert (
    sb_select_account_by_handle(handle=account_handle_2, supabase_client=hub)
    is not None
)
assert account is None

## Clean up

In [ ]:
delete_account(account_handle_1, access_token_1)
delete_account(account_handle_2, access_token_2)

In [ ]:
clean_ci()